In [1]:
import sys
import os

# Chemin absolu ou relatif vers le dossier racine du projet (qui contient le dossier geocharge)
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

In [2]:
from geocharge.preprocessing import load_data, clean_data, parse_coords
from geocharge.regions import extract_departement, filter_by_ville
from geocharge.analysis import count_stations
from geocharge.visualization import generate_map, generate_heatmap

### 1. Fonction de pré-nettoyage (à exécuter en premier)

In [3]:
def preclean_address(adresse: str) -> str:
    """Nettoyage approfondi des adresses avant traitement"""
    if pd.isna(adresse):
        return ""
    
    # Conversion unicode et suppression caractères spéciaux
    adresse = adresse.encode('utf-8', 'ignore').decode('utf-8')
    adresse = re.sub(r'[^\w\s\-\'\/,]', ' ', adresse, flags=re.UNICODE)
    
    # Normalisation des apostrophes et tirets
    adresse = re.sub(r'[\‘\’\`\´]', "'", adresse)
    adresse = re.sub(r'[\–\—]', '-', adresse)
    
    # Suppression des motifs parasites
    patterns_to_remove = [
        r'CEDEX?\b.*$', r'BP \d+', r'\(\d+\)', 
        r'\bLIEU-DIT\b', r'\bZ[AI][C]\b', r'\bFRANCE\b',
        r'\b\d{4,5}\s+\d{4,5}\b'  # Codes postaux dupliqués
    ]
    for pattern in patterns_to_remove:
        adresse = re.sub(pattern, '', adresse, flags=re.IGNORECASE)
    
    # Normalisation des espaces
    adresse = re.sub(r'\s+', ' ', adresse).strip()
    
    return adresse.upper()

In [7]:
import pandas as pd
import re

df = load_data("../data/raw/consolidation-etalab-schema-irve-statique-v-2.3.1-20250608.csv")

# Supposons que la colonne contenant les adresses s'appelle 'adresse_station'
df["adresse_station_clean"] = df["adresse_station"].apply(preclean_address)


c:\code\projets_perso\projets\GeoChargeAnalytics\geocharge\preprocessing.py:9: DtypeWarning: Columns (12,18,19,20,21,22,24,29,33) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(filepath)


In [9]:
# Export vers fichier Excel pour exploration visuelle
# Liste des colonnes à exporter
colonnes_a_exporter = [
    "adresse_station_clean",
    "adresse_station",
    "nom_station",
    "coordonneesXY",
    "nbre_pdc",
    "puissance_nominale",
    "code_insee_commune",
]

# Extraction des colonnes
df_export = df[colonnes_a_exporter].copy()

# Export vers un fichier Excel
df_export.to_excel("../data/processed/exploration_adresses_stations.xlsx", index=False)